In [379]:
!pip install esgf-pyclient --progress-bar off

!pip install pytest --progress-bar off
 
!pip install flake8 --progress-bar off
 
!pip install sphinx --progress-bar off
 
!pip install nbsphinx --progress-bar off
 
!pip install pandoc --progress-bar off
 
!pip install ipykernel --progress-bar off
 
!pip install nbconvert --progress-bar off
 
!pip install defusedxml --progress-bar off
 
!pip install jinja2 --progress-bar off
 
!pip install requests --progress-bar off
 
!pip install webob --progress-bar off
 
!pip install myproxyclient --progress-bar off
 
!pip install nbval --progress-bar off

!pip install requests_cache==0.5 --progress-bar off

!pip install xarray --progress-bar off

In [380]:
from pyesgf.search import SearchConnection
import os
import pandas as pd
import requests
from tqdm import tqdm

In [381]:
from pyesgf.search import SearchConnection
import os
import pandas as pd
import requests
from tqdm import tqdm

import datetime as dt
import numpy as np
import pandas as pd
import re
from matplotlib import pyplot as plt
from netCDF4 import Dataset, date2index, num2date, date2num
import xarray as xr

from shapely.geometry import Point
from shapely.geometry.polygon import Polygon
from netCDF4 import Dataset, date2index, num2date, date2num

In [383]:
conn = SearchConnection('https://esgf-data.dkrz.de/esg-search', distrib=True)
query = conn.new_context(
    latest=True,
    project='CMIP5',
    model='HadCM3',
    experiment='rcp45',
    variable='pr,tasmax,tasmin',
    time_frequency="day",
    realm='atmos',
    ensemble='r1i1p1',
    data_node='esgf-data1.ceda.ac.uk')
results = query.search()

print('len(results)', len(results))
print('variables', results[0].json['variable'])


len(results) 1
variables ['huss', 'pr', 'psl', 'sfcWind', 'ta', 'tas', 'tasmax', 'tasmin', 'ua', 'va', 'zg']


In [384]:
hit = results[0].file_context().search()
print('len(hit)', len(hit))

allfiles = map(lambda f : {'filename': f.filename, 'url': f.download_url}, hit)
allfiles = list(allfiles)
print('len(allfiles)', len(allfiles))


len(hit) 22
len(allfiles) 22


In [385]:
files = []
for variable in ['tasmin', 'tasmax', 'pr']:
  result = list(filter(lambda x: variable in x['filename'], allfiles))
  files.extend(result)

files = pd.DataFrame.from_dict(files)
files

,filename,url
0,tasmin_day_HadCM3_rcp45_r1i1p1_20060101-203012...,http://esgf-data1.ceda.ac.uk/thredds/fileServe...
1,tasmin_day_HadCM3_rcp45_r1i1p1_20310101-203512...,http://esgf-data1.ceda.ac.uk/thredds/fileServe...
2,tasmax_day_HadCM3_rcp45_r1i1p1_20060101-203012...,http://esgf-data1.ceda.ac.uk/thredds/fileServe...
3,tasmax_day_HadCM3_rcp45_r1i1p1_20310101-203512...,http://esgf-data1.ceda.ac.uk/thredds/fileServe...
4,pr_day_HadCM3_rcp45_r1i1p1_20060101-20301230.nc,http://esgf-data1.ceda.ac.uk/thredds/fileServe...
5,pr_day_HadCM3_rcp45_r1i1p1_20310101-20351230.nc,http://esgf-data1.ceda.ac.uk/thredds/fileServe...


In [386]:
 #drop the long-range (2101–2300) projections from our list of downloads
files = files[~files['filename'].str.contains("210101")]

#remove duplicate filenames
files = files.drop_duplicates(subset='filename', keep="first") 

files

,filename,url
0,tasmin_day_HadCM3_rcp45_r1i1p1_20060101-203012...,http://esgf-data1.ceda.ac.uk/thredds/fileServe...
1,tasmin_day_HadCM3_rcp45_r1i1p1_20310101-203512...,http://esgf-data1.ceda.ac.uk/thredds/fileServe...
2,tasmax_day_HadCM3_rcp45_r1i1p1_20060101-203012...,http://esgf-data1.ceda.ac.uk/thredds/fileServe...
3,tasmax_day_HadCM3_rcp45_r1i1p1_20310101-203512...,http://esgf-data1.ceda.ac.uk/thredds/fileServe...
4,pr_day_HadCM3_rcp45_r1i1p1_20060101-20301230.nc,http://esgf-data1.ceda.ac.uk/thredds/fileServe...
5,pr_day_HadCM3_rcp45_r1i1p1_20310101-20351230.nc,http://esgf-data1.ceda.ac.uk/thredds/fileServe...


In [388]:
# Adapted from: https://stackoverflow.com/a/37573701
def download(url, filename):
    print("Downloading ", filename)
    r = requests.get(url, stream=True)
    total_size, block_size = int(r.headers.get('content-length', 0)), 1024
    with open(filename, 'wb') as f:
        for data in tqdm(r.iter_content(block_size), total=total_size//block_size, unit='KiB', unit_scale=True):
            f.write(data)
            
    if total_size != 0 and os.path.getsize(filename) != total_size:
        print("Downloaded size does not match expected size!\n",
              "FYI, the status code was ", r.status_code)

In [389]:
download(files.url[0], files.filename[0])

247kKiB [00:24, 10.1kKiB/s]                         


In [390]:
nc_lat = nc.variables['lat'][:].data
nc_lon = nc.variables['lon'][:].data

In [391]:
nc = Dataset(files.filename[0])

time_start = date2index(dt.datetime(1981, 1, 1), nc_time, select="nearest")
time_end = date2index(dt.datetime(2100, 12, 31), nc_time, select="nearest")

num2date(nc_time[time_end], units=nc_time.units, calendar=nc_time.calendar)

nc_lat = nc.variables['lat'][:].data
nc_lon = nc.variables['lon'][:].data

bangui = pd.read_excel('Bangui_pts.xlsx', index_col=0, dtype={'id': int, 'lat': float, 'long': float})
banguiValues = []
for row in bangui.iterrows():
  banguiValues.append([row[1].long, row[1].lat])
  
polygonBangui = Polygon(banguiValues)

In [409]:
import xarray as xr
ds = xr.open_dataset(files.url[0])
print(ds)

OSError: ignored

In [407]:
nc = Dataset(files.filename[0])
nc.dimensions

{'bnds': <class 'netCDF4._netCDF4.Dimension'>: name = 'bnds', size = 2,
 'lat': <class 'netCDF4._netCDF4.Dimension'>: name = 'lat', size = 73,
 'lon': <class 'netCDF4._netCDF4.Dimension'>: name = 'lon', size = 96,
 'time': <class 'netCDF4._netCDF4.Dimension'> (unlimited): name = 'time', size = 9000}

In [413]:
nc_lat = nc.variables['lat'][:].data
len(nc_lat)

73

In [412]:
nc_lon = nc.variables['lon'][:].data
len(nc_lon)

96

In [ ]:
nc_lat = nc.variables['lat'][:].data
lat_cell = np.where((nc_lat[:,0] < row[1].lat) & (nc_lat[:,1] > row[1].lat))[0][0]

nc_lon = nc.variables['lon'][:].data
lon_cell = np.where(polygonBangui.contains(Point(row[1].long, row[1].long)) (nc_lon[:,0] < row[1].long) & (nc_lon[:,1] > row[1].long))[0][0]

dat = nc.variables['tasmin'][time_start:(time_end + 1), lat_cell, lon_cell]
dat = np.subtract(dat.data, 273.15)
dates = num2date(nc_time[time_start:time_end + 1], units=nc_time.units, calendar=nc_time.calendar)
df = pd.DataFrame({'Date': dates, 'Tmin': dat})
df.head()

In [361]:
nc_lat_bnds = nc.variables["lat_bnds"][:].data
lat_cell = np.where((nc_lat_bnds[:,0] < row[1].lat) & (nc_lat_bnds[:,1] > row[1].lat))[0][0]

nc_lon_bnds = nc.variables["lon_bnds"][:].data
lon_cell = np.where((nc_lon_bnds[:,0] < row[1].long) & (nc_lon_bnds[:,1] > row[1].long))[0][0]

dat = nc.variables['tasmin'][time_start:(time_end + 1), lat_cell, lon_cell]
dat = np.subtract(dat.data, 273.15)
dates = num2date(nc_time[time_start:time_end + 1], units=nc_time.units, calendar=nc_time.calendar)
df = pd.DataFrame({'Date': dates, 'Tmin': dat})
df.head()

KeyError: ignored

In [348]:
df.head()

,Date,MeanTemp
0,2006-01-01 12:00:00,17.978607
1,2006-01-02 12:00:00,17.908112
2,2006-01-03 12:00:00,17.334320
3,2006-01-04 12:00:00,18.653046
4,2006-01-05 12:00:00,17.960724
